# PyTorch Cats vs Dogs Classifier

## import libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import import_ipynb
from SimpleCNN import SimpleCNN

# Import constants defined in another file
import Constants

## 1. Transformations

In [11]:
tranform = {
    'train': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])
}

## 2. Load datasets

In [12]:
data_dir = '../data'
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transform=tranform[x])
    for x in ['train', 'val']
}

dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True)
    for x in ['train', 'val']
}

class_names = image_datasets['train'].classes

## 3. Define a simple Convolutional Neural Network (CNN)

Convolutional Neural Networks (CNNs) are deep learning models designed to process data with a grid-like topology such as images. They are the foundation for most modern computer vision applications to detect features within visual data.

In [13]:
# class SimpleCNN(nn.Module):
#     def __init__(self):
#         super(SimpleCNN, self).__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(3, 16, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.MaxPool2d(2),
#             nn.Conv2d(16, 32, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )
#         self.classifier = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(32 * 32 * 32, 64),
#             nn.ReLU(),
#             nn.Linear(64, 2)
#         )

#     def forward(self, x):
#         x = self.features(x)
#         x = self.classifier(x)
#         return x

model = SimpleCNN()

## 4. Loss and optimizer

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 5. Training loop

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(6):  #use more epochs for better performance
    print(f"Eposh {epoch+1}")
    model.train()
    running_loss = 0
    for images, labels in dataloaders['train']:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Train Loss: {running_loss:4f}')
        

Eposh 1


d:\MyProjects\PyTorchImageClassifierDemo\.venv\Lib\site-packages\PIL\TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Train Loss: 791.408473
Eposh 2


## 6. Evaluation on validation set

In [9]:
model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in dataloaders['val']:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct /total:2f}%')

Validation Accuracy: 79.151661%


## Test the model on a single image

In [10]:
from PIL import Image

def predict_image(img_path):
    image = Image.open(img_path)
    image = tranform['val'](image).unsqueeze(0).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    print(f"Prediction: {class_names[predicted.item()]}")

predict_image("data/predict/dogs/set2-55.jpg")

Prediction: dogs


## Save the model

In [13]:
torch.save(model.state_dict(), 'models/catsvsDogsTorchModel.pth')